In [1]:
import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import scipy
import requests
from imdbpie import Imdb
import nltk
import matplotlib.pyplot as plt
import urllib2
from bs4 import BeautifulSoup
import nltk
import collections
import re
import csv

%matplotlib inline

In [2]:
imdb = Imdb()
imdb = Imdb(anonymize=True)

In [2]:
#imdb.top_250()

In [4]:
movies = []
for dct in imdb.top_250():
    del dct['image']
    movies.append(dct)

In [6]:
Can_rate = []
Vote = []
Rate = []
ID = []
Year = []
Title = []
Type = []
for dct in movies:
    Vote.append(dct['num_votes'])
    Rate.append(dct['rating'])
    ID.append(dct['tconst'])
    Year.append(dct['year'])
    Title.append(dct['title'])
    Can_rate.append(dct['can_rate'])
    Type.append(dct['type'])

In [7]:
df = pd.DataFrame({
        "ID": ID,
        'Title': Title,
        'Year': Year,
        'Vote': Vote,
        'Rate': Rate,
        "Can_rate": Can_rate,
        'Type': Type
    })

In [8]:
df.Type.value_counts()

feature    250
Name: Type, dtype: int64

In [9]:
df.Can_rate.value_counts()

True    250
Name: Can_rate, dtype: int64

In [10]:
# Delete 'Can_rate' and 'Type' column coz all rows are same
df.drop('Can_rate',axis=1,inplace=True)
df.drop('Type',axis=1,inplace=True)

In [101]:
#### To save Top_250 movies into csv
df.to_csv('top_movies.csv',encoding='utf-8')

In [66]:
id_list = list(df.ID)

In [70]:
comments = [[] for i in range(250)]
author_score = [[] for i in range(250)]

In [72]:
for i in id_list:
    url = 'http://www.imdb.com/title/' + i + '/' + 'reviews?start=0' 
    contents = BeautifulSoup(urllib2.urlopen(url),"lxml")
    content = contents.find('div',attrs={'class':'reviews'})
    text = contents.find('td',attrs={'align':'right'}).getText()
    n = int(text[1:text.index('r')].strip()) + 10
    for p in range(0,n,10):
        # comments
        url = 'http://www.imdb.com/title/' + i + '/' + 'reviews?start=' + str(p) 
        contents = BeautifulSoup(urllib2.urlopen(url),"lxml")
        content = contents.find('div',attrs={'id':'tn15content'})
        review = content.find_all('p')
        for rew in review:
            comments[id_list.index(i)].append(rew.text) 
        # uid and stars
        
        for user in content.find_all('div'):
            for a in user.find_all('a',href=True)[:1]:
                author_score[id_list.index(i)].append(a.get("href"))
                for img in user.find_all('img', alt=True):
                    author_score[id_list.index(i)].append(img.get('alt'))


    print '# '+str(id_list.index(i)+1) + ' finished'


# 1 finished
# 2 finished
# 3 finished
# 4 finished
# 5 finished
# 6 finished
# 7 finished
# 8 finished
# 9 finished
# 10 finished
# 11 finished
# 12 finished
# 13 finished
# 14 finished
# 15 finished
# 16 finished
# 17 finished
# 18 finished
# 19 finished
# 20 finished
# 21 finished
# 22 finished
# 23 finished
# 24 finished
# 25 finished
# 26 finished
# 27 finished
# 28 finished
# 29 finished
# 30 finished
# 31 finished
# 32 finished
# 33 finished
# 34 finished
# 35 finished
# 36 finished
# 37 finished
# 38 finished
# 39 finished
# 40 finished
# 41 finished
# 42 finished
# 43 finished
# 44 finished
# 45 finished
# 46 finished
# 47 finished
# 48 finished
# 49 finished
# 50 finished
# 51 finished
# 52 finished
# 53 finished
# 54 finished
# 55 finished
# 56 finished
# 57 finished
# 58 finished
# 59 finished
# 60 finished
# 61 finished
# 62 finished
# 63 finished
# 64 finished
# 65 finished
# 66 finished
# 67 finished
# 68 finished
# 69 finished
# 70 finished
# 71 finished
# 72 finished
#

error: [Errno 54] Connection reset by peer

In [9]:
###### To delete "*** This review may contain spoilers ***" and 'Add another review' in comments
reviews = [[] for i in range(250)]

for comment in comments:
    for s in comment:
        if s != "*** This review may contain spoilers ***" and s != 'Add another review':
            reviews[comments.index(comment)].append(s)
        

In [15]:
##### To split author_score into uid and star 
uid = [[] for i in range(250)]
star = [[] for i in range(250)]

for uss in author_score:
    for i in uss:
        try:
            if i.startswith('/user/') and uss[uss.index(i)+1].startswith('/user/'):
                uid[author_score.index(uss)].append(i)
                star[author_score.index(uss)].append('None')
            elif (i.startswith('/user/')) and uss[uss.index(i)+1].endswith('/10'):
                uid[author_score.index(uss)].append(i)
                star[author_score.index(uss)].append(uss[uss.index(i)+1])
        except IndexError: # catch the error
            uid[author_score.index(uss)].append('error')
            star[author_score.index(uss)].append('error')
            continue

In [19]:
import pickle
with open('reviews','wb') as f:
    pickle.dump(reviews,f)

In [20]:
import pickle
with open('uid','wb') as f:
    pickle.dump(uid,f)

In [21]:
import pickle
with open('star','wb') as f:
    pickle.dump(star,f)

### To transfer data into Pandas DataFrame

In [23]:
import pickle
with open('star','rb') as f:
    star = pickle.load(f)

In [26]:
import pickle
with open('uid','rb') as f:
    uid = pickle.load(f)

In [28]:
import pickle
with open('reviews','rb') as f:
    reviews = pickle.load(f)

In [35]:
df = pd.read_csv('top_movies.csv')
id_list = list(df.ID)

In [36]:
# To build star dataframe
total = []
for i in id_list:
    df = pd.DataFrame({i: star[id_list.index(i)]})
    total.append(df)
star = pd.concat(total, axis=1)

In [37]:
star

,tt0111161,tt0068646,tt0071562,tt0468569,tt0050083,tt0108052,tt0110912,tt0167260,tt0060196,tt0137523,...,tt0046250,tt0058461,tt0092067,tt2015381,tt0083922,tt1504320,tt0848228,tt0087884,tt0101414,tt0169102
0,10/10,10/10,10/10,10/10,10/10,10/10,10/10,10/10,10/10,None,...,None,9/10,10/10,10/10,10/10,10/10,1/10,10/10,10/10,10/10
1,10/10,10/10,10/10,10/10,10/10,10/10,10/10,10/10,None,None,...,8/10,None,10/10,10/10,10/10,10/10,None,10/10,10/10,10/10
2,None,10/10,10/10,10/10,9/10,10/10,10/10,10/10,10/10,10/10,...,None,None,10/10,9/10,10/10,8/10,10/10,10/10,10/10,None
3,10/10,10/10,10/10,10/10,10/10,10/10,10/10,10/10,10/10,10/10,...,None,9/10,9/10,9/10,10/10,10/10,10/10,None,9/10,None
4,None,10/10,10/10,10/10,None,None,10/10,10/10,10/10,10/10,...,10/10,10/10,10/10,10/10,10/10,10/10,10/10,10/10,None,None
5,8/10,9/10,10/10,10/10,10/10,None,None,None,10/10,10/10,...,9/10,None,10/10,10/10,9/10,10/10,None,10/10,9/10,9/10
6,10/10,10/10,9/10,10/10,9/10,None,10/10,10/10,10/10,10/10,...,None,9/10,10/10,10/10,10/10,None,None,None,9/10,8/10
7,10/10,10/10,10/10,10/10,None,10/10,None,9/10,10/10,10/10,...,9/10,8/10,10/10,9/10,5/10,10/10,9/10,None,None,None
8,10/10,10/10,10/10,10/10,8/10,10/10,10/10,None,10/10,10/10,...,9/10,7/10,10/10,1/10,9/10,10/10,10/10,None,None,9/10
9,10/10,10/10,None,10/10,None,9/10,None,None,10/10,9/10,...,10/10,8/10,10/10,9/10,None,10/10,None,8/10,10/10,8/10


In [40]:
# To build uid dataframe
total = []
for i in id_list:
    df = pd.DataFrame({i: uid[id_list.index(i)]})
    total.append(df)
uid = pd.concat(total, axis=1)

In [42]:
def fff(x):
    try:
        if np.isnan(x):
            return
    except:
        return x[8:-1]
uid = uid.applymap(fff)

In [43]:
uid

,tt0111161,tt0068646,tt0071562,tt0468569,tt0050083,tt0108052,tt0110912,tt0167260,tt0060196,tt0137523,...,tt0046250,tt0058461,tt0092067,tt2015381,tt0083922,tt1504320,tt0848228,tt0087884,tt0101414,tt0169102
0,2509775,5445669,1170042,3789453,0599605,23582121,0556667,0357354,1125206,1109119,...,0185415,3483002,0496040,23582121,2395638,23873178,24141742,1085885,1293485,0508040
1,0997166,1241331,0200644,12680645,0668851,0676300,1515595,1931776,3844483,2208897,...,1406078,0821456,3924052,33229670,0453068,7243054,5677923,0909672,0290901,2488512
2,1005460,0218695,3782370,19572672,4070806,0671533,0059014,2955724,0134559,0104423,...,1318093,1346534,4336942,14299499,1325213,19511193,27409926,0270026,0312873,2177694
3,1898687,0453228,0588991,2862626,1869824,0609312,4568292,2919262,0314962,0115633,...,0137908,0312444,4589081,42701159,1124558,24040984,6965396,1814173,3379318,1222869
4,0094239,5347756,0176092,13846894,0643062,0355979,4332836,1142410,5890948,0863417,...,7074179,0408624,4337237,21941782,1098460,2670273,27750781,18207359,1048771,2028078
5,1285640,0078412,8564485,2371300,2011184,0478810,0688559,1002035,0054006,0328956,...,0176092,0381265,0889372,22640195,0954062,13724283,3342100,0517315,0785249,3313572
6,0611718,7154267,0139258,10583368,0645286,0823877,15794099,2362433,1052098,0473523,...,1174211,0176092,0378876,26519067,0012295,1391596,6143489,3432480,2348273,0176092
7,0257957,11305442,3341647,2747265,1318549,9295399,0670675,0064493,0179458,15794099,...,0989035,2488512,4277036,24740649,19544182,7339516,1497432,0923228,0220605,0351219
8,0355122,24740649,7484735,1293485,2159966,0034784,0453068,1606763,0090865,0522512,...,2483625,0279681,4417949,51230348,1070041,18460258,24740649,2420383,1002035,1225869
9,0842118,3920272,0167590,5237888,0688559,11178723,2372297,0668958,0453068,0908124,...,0583640,0111731,0677266,48490287,1656172,5069285,5327796,3711290,0668626,2467654


In [44]:
# To build review dataframe
total = []
for i in id_list:
    df = pd.DataFrame({i: reviews[id_list.index(i)]})
    total.append(df)
reviews = pd.concat(total, axis=1)

In [45]:
reviews

,tt0111161,tt0068646,tt0071562,tt0468569,tt0050083,tt0108052,tt0110912,tt0167260,tt0060196,tt0137523,...,tt0046250,tt0058461,tt0092067,tt2015381,tt0083922,tt1504320,tt0848228,tt0087884,tt0101414,tt0169102
0,\nWhy do I want to write the 234th comment on ...,\nRather than concentrating on everything that...,\n\nThis movie is way to be good to be labelle...,\nWe've been subjected to enormous amounts of ...,\n\nAn excellent courtroom drama with a unique...,"\nI know many of you, out there will disagree ...","\nOne of the early scenes in ""Pulp Fiction"" fe...",\n\nPeter Jackson has done it. He has created...,\n\nA sprawling Western epic that follows the ...,\n\nWhen I first saw the previews for this mov...,...,\n\nThis charming comedy is justly famous as t...,\nALthough in many respects this film pales in...,\n\nHayao Miyazaki has no equal when it comes ...,\nGuardians of the Galaxy is nothing short of ...,\n\nMost of the ideas revealed through mystery...,"\nThis is a biopic about how King George VI, t...",\nHave movies become so bad these days that a ...,\n\nI first saw this film almost fifteen years...,\nBeauty and the Beast is without a doubt one ...,"\n\nOkay, I am not from India, not even close...."
1,"\n\nCan Hollywood, usually creating things for...",\n\nThis is by far the best movie ever to give...,\n\nThe original Godfather is a brilliant work...,"\nChristopher Nolan's second bundle of joy ""Th...","\n\nThis film is superb, in fact as Shakespear...",\n\nMany movies come out each year and we appl...,"\nTo put this in context, I am 34 years old an...",\n\nSaying that this film starts where `Two To...,"\n\nThis is the third,and arguably the best, o...","\n\nI am, unfortunately, not one of the faithf...",...,\nI recently caught this little gem of a film ...,"\n\nWhen Per un pungo di dollari, or A Fistful...",\n\nI have seen just about all of Miyazaki's f...,\nI entitle my review A New Hope as I walk out...,"\nAs Ingmar Bergman's ""swan song"" (which wasn'...",\nI think I must have seen a different film fr...,\nI have got to tell anyone who will listen th...,\n\nIt's hard for me to select just one movie ...,\n\nThe only animated movie to receive a Best ...,"\nIn 1893, India was a colony of England. The ..."
2,\n\nI have never seen such an amazing film sin...,\n\nThe Godfather is one of the few films in w...,\n\nYou can count on one hand the movie sequel...,\nI had the honor of watching TDK during a scr...,\n'12 Angry Men' is an outstanding film. It is...,\n\nSchindler's List is not only a pure master...,"\n\nIf you think ""Pulp Fiction"" is brilliant, ...","\nI am, I admit, an unlikely convert to the re...",\n\nThis is without a doubt my all-time favori...,\n\nFight Club is one of the most unique films...,...,\n\nA plot as slender as Audrey Hepburn but oh...,\n\nClint Eastwood was best known to American ...,\nThis is one of the best animated family film...,\nI remember sitting in Hall H at Comic-Con tw...,\nSwedish director Ingmar Bergman has a reputa...,\nThere were a lot of elderly folks in the the...,\nI'm sorry to say The Avengers isn't a good m...,"\n\nIt might seem odd to call this an ""America...",\n\na great movie story done with wondrous ani...,\n\nThis movie not only opened my eyes to the...
3,"\nIn its Oscar year, Shawshank Redemption (wri...","\n\n This movie is strong, good script, gr...",\n\nThe Godfather Part 2 is the finest sequel ...,\nI used to leave a theatre after seeing a hig...,\n........Films rarely get this uplifting and ...,\n\nThe first (and last) time I viewed this ma...,\nPulp Fiction may be the single best film eve...,"\n\nAs a movie watcher, I tend to become bored...","\n\nOk- first, as mentioned in another review,...","\n\n""Fight Club"" an aggressive, confrontationa...",...,\n\nAudrey Hepburn's first big film role cast ...,\n\n'A Fistful of Dollars' is the first from S...,\nI first saw this film when I was about 8 yea...,\nExpectations: surpassed - and they were pret...,\n\nI have wanted to see 

In [46]:
uid.to_csv('./uid.csv')
star.to_csv('./star.csv')
reviews.to_csv('./reviews.csv', encoding='utf-8')

## To Create dataframe with UID MovieID Rating columns

In [86]:
top = pd.read_csv('top_movies.csv')
id_list = list(top.ID)

uid = pd.read_csv('uid.csv')
star = pd.read_csv('star.csv')
uid = uid.iloc[:,1:]

"""
reviews = pd.read_csv('reviews.csv')
reviews = reviews.iloc[:,1:]
"""
#Convert 10/10, 8/10 .... to 10 and 8...
def fff(x):
    try:
        if x.endswith('/10'):
            return int(x[:x.find('/')])
    except:
        return
star = star.iloc[:,1:].applymap(fff)

In [63]:
5069*250

1267250

In [87]:
# To create MID_UID dataframe
complex_table = pd.DataFrame(columns = ['MID','UID'])
tables = []
for i in id_list:
    table = pd.melt(uid,value_vars=[i],var_name = 'MID', value_name = 'UID')
    tables.append(table)
MID_UID = complex_table.append(tables,ignore_index=True)

In [88]:
MID_UID

,MID,UID
0,tt0111161,2509775.0
1,tt0111161,997166.0
2,tt0111161,1005460.0
3,tt0111161,1898687.0
4,tt0111161,94239.0
5,tt0111161,1285640.0
6,tt0111161,611718.0
7,tt0111161,257957.0
8,tt0111161,355122.0
9,tt0111161,842118.0


In [89]:
len(MID_UID['UID'].unique())

111007

In [90]:
# To create MID_STAR dataframe
complex_table = pd.DataFrame(columns = ['MID','STAR'])
tables = []
for i in id_list:
    table = pd.melt(star,value_vars=[i],var_name = 'MID', value_name = 'STAR')
    tables.append(table)
MID_STAR = complex_table.append(tables,ignore_index=True)

In [91]:
MID_STAR

,MID,STAR
0,tt0111161,10.0
1,tt0111161,10.0
2,tt0111161,NaN
3,tt0111161,10.0
4,tt0111161,NaN
5,tt0111161,8.0
6,tt0111161,10.0
7,tt0111161,10.0
8,tt0111161,10.0
9,tt0111161,10.0


In [92]:
# To create MID_STAR_UID dataframe
result = pd.concat([MID_STAR,MID_UID],axis=1,ignore_index=True)
result.columns = ['MID','STAR','MID2','UID']
result.drop('MID2',axis=1,inplace=True)
result

,MID,STAR,UID
0,tt0111161,10.0,2509775.0
1,tt0111161,10.0,997166.0
2,tt0111161,NaN,1005460.0
3,tt0111161,10.0,1898687.0
4,tt0111161,NaN,94239.0
5,tt0111161,8.0,1285640.0
6,tt0111161,10.0,611718.0
7,tt0111161,10.0,257957.0
8,tt0111161,10.0,355122.0
9,tt0111161,10.0,842118.0


In [93]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267250 entries, 0 to 1267249
Data columns (total 3 columns):
MID     1267250 non-null object
STAR    175914 non-null float64
UID     205708 non-null float64
dtypes: float64(2), object(1)
memory usage: 29.0+ MB


In [94]:
# To delete NAN in UID column
result = result[result['UID'].isnull() == False]
result

,MID,STAR,UID
0,tt0111161,10.0,2509775.0
1,tt0111161,10.0,997166.0
2,tt0111161,NaN,1005460.0
3,tt0111161,10.0,1898687.0
4,tt0111161,NaN,94239.0
5,tt0111161,8.0,1285640.0
6,tt0111161,10.0,611718.0
7,tt0111161,10.0,257957.0
8,tt0111161,10.0,355122.0
9,tt0111161,10.0,842118.0


In [122]:
# Fill NAN with mean in each group

MID_STAR_UID = result.groupby('MID').transform(lambda x: x.fillna(x.mean()))

In [123]:
# Add MID column back and change 'STAR' 'UID' type
MID_STAR_UID = pd.concat([MID_STAR_UID,result[['MID']]], axis = 1)
MID_STAR_UID['STAR'] = MID_STAR_UID['STAR'].astype(int)
MID_STAR_UID['UID'] = MID_STAR_UID['UID'].astype(int)
MID_STAR_UID

,STAR,UID,MID
0,10,2509775,tt0111161
1,10,997166,tt0111161
2,9,1005460,tt0111161
3,10,1898687,tt0111161
4,9,94239,tt0111161
5,8,1285640,tt0111161
6,10,611718,tt0111161
7,10,257957,tt0111161
8,10,355122,tt0111161
9,10,842118,tt0111161


In [124]:
MID_STAR_UID.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205708 entries, 0 to 1262428
Data columns (total 3 columns):
STAR    205708 non-null int64
UID     205708 non-null int64
MID     205708 non-null object
dtypes: int64(2), object(1)
memory usage: 6.3+ MB


In [128]:
pivot = MID_STAR_UID.pivot_table(values='STAR',index='UID',columns='MID')
pivot

MID,tt0012349,tt0015864,tt0017136,tt0017925,tt0018455,tt0019254,tt0021749,tt0022100,tt0025316,tt0027977,...,tt2096673,tt2106476,tt2267998,tt2278388,tt2488496,tt2582802,tt2948356,tt3011894,tt3170832,tt3659388
UID,,,,,,,,,,,,,,,,,,,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
print "Number of reviews: " + str(len(MID_STAR_UID))
print "Number of users: " + str(pivot.index.shape[0])
print "Number of movies: " + str(len(pivot.columns))

Number of reviews: 205708
Number of users: 111006
Number of movies: 250


In [133]:
MID_STAR_UID.to_csv('MID_STAR_UID.csv')

In [134]:
pivot.to_csv('pivot.csv')


## Cosine similarity

In [2]:
df = pd.read_csv('pivot.csv',index_col='UID')
df.fillna(0,inplace=True)
df

,tt0012349,tt0015864,tt0017136,tt0017925,tt0018455,tt0019254,tt0021749,tt0022100,tt0025316,tt0027977,...,tt2096673,tt2106476,tt2267998,tt2278388,tt2488496,tt2582802,tt2948356,tt3011894,tt3170832,tt3659388
UID,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df_bigger_50 = df[df.sum(axis=1)>50]
df_bigger_50

,tt0012349,tt0015864,tt0017136,tt0017925,tt0018455,tt0019254,tt0021749,tt0022100,tt0025316,tt0027977,...,tt2096673,tt2106476,tt2267998,tt2278388,tt2488496,tt2582802,tt2948356,tt3011894,tt3170832,tt3659388
UID,,,,,,,,,,,,,,,,,,,,,
1220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3136,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11762,0.0,0.0,0.0,7.0,0.0,7.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0
12098,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_bigger_50 = df_bigger_50.unstack().reset_index(name='value')

In [5]:
# all stars including 0 from 3237 users (3237*250)
df_bigger_50

,level_0,UID,value
0,tt0012349,1220,0.0
1,tt0012349,2746,0.0
2,tt0012349,3136,0.0
3,tt0012349,5684,0.0
4,tt0012349,9034,0.0
5,tt0012349,9605,0.0
6,tt0012349,11596,0.0
7,tt0012349,11762,0.0
8,tt0012349,12098,0.0
9,tt0012349,12295,0.0


In [6]:
# delete all 0 stars
df = df_bigger_50[df_bigger_50['value'] != 0]

In [8]:
df = df.reset_index(drop=True)

In [10]:
df.columns = ['MID','UID','value']

In [11]:
df

,MID,UID,value
0,tt0012349,88197,10.0
1,tt0012349,137908,8.0
2,tt0012349,222418,8.0
3,tt0012349,453068,10.0
4,tt0012349,562732,10.0
5,tt0012349,605876,8.0
6,tt0012349,679729,10.0
7,tt0012349,819382,8.0
8,tt0012349,826090,8.0
9,tt0012349,850869,10.0


In [12]:
mid_unique = df['MID'].unique().tolist()
movie_menu = np.arange(1,251)
mid_seq = []
for i in df['MID']:
    ind = mid_unique.index(i)
    mid_seq.append(movie_menu[ind])

In [13]:
df['MID_seq'] = mid_seq

In [14]:
uid_unique = df['UID'].unique().tolist()
uid_unique.sort()

In [15]:
user_menu = np.arange(1,111007)

In [16]:
uid_seq = []
for i in df['UID']:
    ind = uid_unique.index(i)
    uid_seq.append(user_menu[ind])

In [17]:
df['UID_seq'] = uid_seq

In [18]:
df

,MID,UID,value,MID_seq,UID_seq
0,tt0012349,88197,10.0,1,57
1,tt0012349,137908,8.0,1,112
2,tt0012349,222418,8.0,1,203
3,tt0012349,453068,10.0,1,426
4,tt0012349,562732,10.0,1,499
5,tt0012349,605876,8.0,1,531
6,tt0012349,679729,10.0,1,587
7,tt0012349,819382,8.0,1,669
8,tt0012349,826090,8.0,1,677
9,tt0012349,850869,10.0,1,687


In [ ]:
# df.to_csv('../MID_UID_seq')

In [19]:
del df['UID']
del df['MID']

In [20]:
df

,value,MID_seq,UID_seq
0,10.0,1,57
1,8.0,1,112
2,8.0,1,203
3,10.0,1,426
4,10.0,1,499
5,8.0,1,531
6,10.0,1,587
7,8.0,1,669
8,8.0,1,677
9,10.0,1,687


In [21]:
print "Unique movie: " + str(df['MID_seq'].unique().shape[0])
print "Unique user: " + str(df['UID_seq'].unique().shape[0])
print "length of dataset: " + str(len(df)) 

Unique movie: 250
Unique user: 3237
length of dataset: 58962


In [22]:
n_users = df.UID_seq.unique().shape[0]

n_items = df.MID_seq.unique().shape[0]

print 'Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items)

Number of users = 3237 | Number of movies = 250


In [23]:
from sklearn import cross_validation as cv

train_data,test_data = cv.train_test_split(df, test_size = 0.25)

In [25]:
train_data_matrix = np.zeros((n_users,n_items))
for line in train_data.itertuples():
    train_data_matrix[line[3]-1, line[2]-1] = line[1]

In [26]:
test_data_matrix = np.zeros((n_users, n_items))

for line in test_data.itertuples():
    test_data_matrix[line[3]-1, line[2]-1] = line[1]

In [27]:
from sklearn.metrics.pairwise import pairwise_distances

user_similarity = pairwise_distances(train_data_matrix, metric = "cosine")

item_similarity = pairwise_distances(train_data_matrix.T, metric = "cosine")

In [28]:
user_similarity.shape

(3237, 3237)

In [29]:
item_similarity.shape

(250, 250)

In [33]:
def predict(rating, similarity, type = 'user'):

    if type == 'user':

        mean_user_rating = rating.mean(axis = 1)

        rating_diff = (rating - mean_user_rating[:,np.newaxis])

        pred = mean_user_rating[:,np.newaxis] + similarity.dot(rating_diff) / np.array([np.abs(similarity).sum(axis=1)]).T

    elif type == 'item':

        pred = rating.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

    return pred

In [34]:
item_prediction = predict(train_data_matrix, item_similarity, type = 'item')

user_prediction = predict(train_data_matrix, user_similarity, type = 'user')

In [35]:
from sklearn.metrics import mean_squared_error

from math import sqrt

def rmse(prediction, ground_truth):

    prediction = prediction[ground_truth.nonzero()].flatten()

    ground_truth = ground_truth[ground_truth.nonzero()].flatten()

    return sqrt(mean_squared_error(prediction, ground_truth))

In [36]:
print 'User based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix))

print 'Item based CF RMSe: ' + str(rmse(item_prediction, test_data_matrix))

User based CF RMSE: 7.45533325543
Item based CF RMSe: 7.60808700642


In [37]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

    #get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 5)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)

print 'User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix))

User-based CF MSE: 6.99606230619


In [38]:
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[3]-1, row[2]-1] = row[1]
ratings

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  7.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [39]:
ratings.nonzero()

(array([   0,    0,    0, ..., 3236, 3236, 3236]),
 array([ 47,  56,  57, ..., 196, 197, 245]))

In [40]:
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print 'Sparsity: {:4.2f}%'.format(sparsity)

Sparsity: 7.29%
